In [1]:
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, GlobalMaxPooling1D
from tensorflow.keras.optimizers import RMSprop


import numpy as np
import random
import re

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5530229594410818595
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3181130547
locality {
  bus_id: 1
  links {
  }
}
incarnation: 319415492416728761
physical_device_desc: "device: 0, name: GeForce GTX 960M, pci bus id: 0000:01:00.0, compute capability: 5.0"
]


In [2]:
# Loading in some Borges. Shakespeare is boring!

with open(f'./borges/borges_collected_fictions.txt', 'r', encoding='utf8') as f:
            data = f.read()
        
print(data[:500])

PENGUIN BOOKS





COLLECTED FICTIONS





Jorge Luis Borges was born in Buenos Aires in 1899 and was educated in Europe. One of the most widely acclaimed writers of our time, he published many collections of poems, essays, and short stories before his death in Geneva in June 1986. In 1961 Borges shared the International Publishers’ prize with Samuel Beckett. The Ingram Merrill Foundation granted him its Annual Literary Award in 1966 for his “outstanding contribution to literature.” In 1971 Colu


In [3]:
def cleaner(text):
    '''cleans the text of escape chars and the like.'''
    stripped = text.strip(' ')
    lowered = stripped.lower()
    cleaned = re.sub(r"[^a-zA-Z ^0-9 \. , ']", ' ', lowered)
    despaced = cleaned.strip(' ')
    return despaced

In [4]:
# clean the data
clean_data = cleaner(data)

# check that things look alright
clean_data[:400]

'penguin books      collected fictions      jorge luis borges was born in buenos aires in 1899 and was educated in europe. one of the most widely acclaimed writers of our time, he published many collections of poems, essays, and short stories before his death in geneva in june 1986. in 1961 borges shared the international publishers  prize with samuel beckett. the ingram merrill foundation granted '

In [5]:
# make a list of all unique words, check how many
wordlist = clean_data.split(' ')
wordset = list(set(clean_data.split(' ')))
len(wordset)

26646

In [6]:
# get rid of words below a certain frequency
freqdict = {word:wordlist.count(word) for word in wordset}

In [7]:
freqdict['i']

3723

In [8]:
smaller_list = [ word for word in wordlist if (freqdict[word] >= 10) and (word != '') ] 
len(set(smaller_list))

2246

In [9]:
small_wordset = list(set(smaller_list))

In [10]:
# encode words as ints

words_to_ints = {w:i for i,w in enumerate(small_wordset)}
ints_to_words = {i:w for i,w in enumerate(small_wordset)}

In [11]:
words_to_ints['and']

1473

In [12]:
# Create the Sequence Data

maxlen = 40
step = 5

encoded = [words_to_ints[w] for w in smaller_list]

sequences = []
next_words = []

for ii in range(0, len(encoded)-maxlen, step):
    sequences.append(encoded[ii : ii+maxlen])
    next_words.append(encoded[ii+maxlen])

In [13]:
# Specify x & y

x = np.zeros((len(sequences), maxlen, len(small_wordset)), dtype=np.bool)
y = np.zeros((len(sequences), len(small_wordset)), dtype=np.bool)

for ii, sequence in enumerate(sequences):
    for t, word in enumerate(sequence):
        x[ii,t,word] = 1
        
    y[ii, next_words[ii]] = 1

In [14]:
X2 = np.asarray(sequences)

In [15]:
y2 = np.asarray(next_words)
y2.shape = (y2.shape[0], 1)

In [16]:
x[1].shape

(40, 2246)

In [17]:
# build the model: a single LSTM

model = Sequential()
model.add(Conv1D(128, kernel_size=7, input_shape=(maxlen, len(small_wordset)), activation='relu', strides=1))
model.add(GlobalMaxPooling1D())
model.add(Dense(len(small_wordset), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')